# Regular 3d fracture network

Extension to 3d of the test case 4.1 from the benchmark study [Flemisch 2017]. Both low permeable and high permeable fractures are considered. The domain is $(0, 1)^3$ and 9 fractures are present, see the `file geiger_3d.csv` for the coordinates. A graphical representation is given by<br>
<img src="geiger_network_0.png" width="50%"/>

In [34]:
import numpy as np
from porepy.fracs import importer

# geometrical tolerance
tol = 1e-8

# define the mesh size
h = 0.1
grid_kwargs = {}
grid_kwargs['mesh_size'] = {'mode': 'constant', 'value': h, 'bound_value': h, 'tol': tol}

# first line in the file is the domain boundary, the others the fractures
file_dfm = 'geiger_3d.csv'
gb, domain = importer.dfm_3d_from_csv(file_dfm, tol, **grid_kwargs)
gb.compute_geometry()

data_problem = {'domain': domain, 'tol': tol, 'aperture': 1e-4, 'km': 1, 'kf': 1e-4}

Use existing intersections
Use existing decomposition
Using old version of mesh size determination
Gmsh processed file successfully


Grid creation completed. Elapsed time 0.179154634475708


Created 1 3-d grids with 9430 cells
Created 9 2-d grids with 1788 cells
Created 69 1-d grids with 159 cells
Created 27 0-d grids with 27 cells




To following lines will be removed in a newer version.

In [30]:
%%capture
from porepy.grids.grid import FaceTag

internal_flag = FaceTag.FRACTURE
[g.remove_face_tag_if_tag(FaceTag.BOUNDARY, internal_flag) for g, _ in gb]

We introduce the class defining most of the data for the problem.

In [31]:
from porepy.params import tensor
from porepy.params.bc import BoundaryCondition
from porepy.numerics import elliptic

#------------------------------------------------------------------------------#

class DarcyModelData(elliptic.EllipticDataAssigner):
    def __init__(self, g, data, **kwargs):
        self.domain = kwargs['domain']
        self.tol = kwargs['tol']

        self.apert = kwargs['aperture']
        self.km = kwargs['km']
        self.kf = kwargs['kf']
        
        # define two pieces of the boundary, useful to impose boundary conditions
        b_face = g.get_domain_boundary_faces()
        self.flux_boundary = np.empty(0)
        self.pressure_boundary = np.empty(0)
        
        if b_face.size > 0:
            b_face_centers = g.face_centers[:, b_face]
            
            val = 0.5 - self.tol
            self.b_flux = np.logical_and.reduce(tuple(b_face_centers[i, :] < val for i in range(3)))
           
            val = 0.75 + self.tol
            self.b_pressure = np.logical_and.reduce(tuple(b_face_centers[i, :] > val for i in range(3)))
        
        elliptic.EllipticDataAssigner.__init__(self, g, data)

    def aperture(self):
        return np.power(self.apert, 3 - self.grid().dim)

    def bc(self):
        bound_faces = self.grid().get_domain_boundary_faces()
        if bound_faces.size == 0:
            return BoundaryCondition(self.grid(), np.empty(0), np.empty(0))

        labels = np.array(['neu'] * bound_faces.size)
        labels[self.b_flux] = 'dir' # aaaaaaaaaaaaaaaaaaaaaaaaaa should be neu as well
        labels[self.b_pressure] = 'dir'

        return BoundaryCondition(self.grid(), bound_faces, labels)

    def bc_val(self):
        bc_val = np.zeros(self.grid().num_faces)
        bound_faces = self.grid().get_domain_boundary_faces()
        if bound_faces.size > 0:
            bc_val[bound_faces[self.b_flux]] = 1
            bc_val[bound_faces[self.b_pressure]] = -1
        
        return bc_val

    def permeability(self):
        if self.grid().dim == 3:
            kxx = self.km * np.ones(self.grid().num_cells)
            return tensor.SecondOrder(self.grid().dim, kxx=kxx, kyy=kxx, kzz=kxx)
        elif self.grid().dim == 2:
            kxx = self.kf * np.ones(self.grid().num_cells)
            return tensor.SecondOrder(self.grid().dim, kxx=kxx, kyy=kxx, kzz=1)
        else: # dim == 1
            kxx = self.kf * np.ones(self.grid().num_cells)
            return tensor.SecondOrder(self.grid().dim, kxx=kxx, kyy=1, kzz=1)

Add the data to the grid bucket and the coupling permeability among objects of different dimensions.

In [32]:
gb.add_node_props(['is_tangential', 'problem', 'frac_num'])
for g, d in gb:
    d['problem'] = DarcyModelData(g, d, **data_problem)
    d['is_tangential'] = True

    if g.dim == 2:
        d['frac_num'] = g.frac_num*np.ones(g.num_cells)
    else:
        d['frac_num'] = -1*np.ones(g.num_cells)        

# Assign coupling permeability, the aperture is read from the lower dimensional grid
gb.add_edge_prop('kn')
for e, d in gb.edges_props():
    g_l = gb.sorted_nodes_of_edge(e)[0]
    aperture = gb.node_prop(g_l, 'param').get_aperture()
    d['kn'] = data_problem['kf'] / aperture

In [33]:
problem = elliptic.DualEllipticModel(gb)
problem.solve()
problem.split()

problem.pressure('pressure')
problem.discharge('discharge')
problem.project_discharge('P0u')
problem.save(['pressure', 'P0u', 'frac_num'])

To convert this script in pure python:

In [18]:
!jupyter nbconvert --to script geiger_3d.ipynb

[NbConvertApp] Converting notebook geiger_3d.ipynb to script
[NbConvertApp] Writing 599 bytes to geiger_3d.py
